In [38]:
import numpy as np
import os
import pandas as pd
import glob
import h5py
import tables
import pickle
from ast import literal_eval
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.optimize import curve_fit

from ast import literal_eval
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score



import psutil
import sys
from multiprocessing import Pool
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.layers import Convolution1D, MaxPooling1D, AveragePooling1D
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import *
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import h5py
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.layers import TimeDistributed

## Preprocessing Data

This script takes raw data, create a pandas df and saves it to disk as a hdf5 file called `data.h5`. Before running this code, make sure your file directory looks like this...

```
.
│   Data Merged.ipynb
│   ...    
│
└───Data
    │   search_grid.csv
    │   impact_score_2017.csv
    │   impact_score_2017.csv
    |   impact_score_2019_public_test.csv
    |
    └───GFS_2017
    |   │   gfs_4_20170101_0000_000.csv
    |   │   ...
    |
    └───GFS_2018
    |   │   gfs_4_20180101_0000_000.csv
    |   │   ...     
    | 
    └───GFS_2019
        │   gfs_4_20190101_0000_000.csv
        │   ...     
   

```

### Loading Provided Data

In [39]:
#### Path to all data files

# All weather
weather17 = ["./Data/GFS_2017/" + f for f in os.listdir("./Data/GFS_2017")]
weather18 = ["./Data/GFS_2018/" + f for f in os.listdir("./Data/GFS_2018")]
weather19 = ["./Data/GFS_2019/" + f for f in os.listdir("./Data/GFS_2019")]
allweather = weather17 + weather18 + weather19

# All impact
allimpact = ["./Data/" + f for f in os.listdir("./Data") if f.startswith('impact')]

# Loc2zip map
grid = "./Data/search_grid.csv"

In [40]:
### Load weather into 1 df
# Concat into a giant dataframe
weather = pd.concat(map(pd.read_csv, allweather), ignore_index=True)

In [41]:
### Load and flatten loc2zip map
loc2zip = pd.read_csv(grid, converters={"mapped_zipcodes": literal_eval})
loc2zip = loc2zip.explode("mapped_zipcodes").reset_index()
loc2zip['mapped_zipcodes'] = pd.to_numeric(loc2zip['mapped_zipcodes'])
#loc2zip.rename(columns={"mapped_zipcodes":"zip5"}, inplace=True)

In [42]:
### Load impact into 1 df
impact = pd.concat(map(pd.read_csv, allimpact), ignore_index=True).drop(["Unnamed: 0"], axis=1)

In [43]:
### Join impact and loc2zip
impact_loc = impact.join(loc2zip.set_index('mapped_zipcodes'), on='zip5').drop(["index"], axis = 1)

In [44]:
### Join impact and weather in loc and date
# Preprocessing on join columns 
impact_loc["date_key"] = pd.to_datetime(impact_loc["date_key"])
weather["Date"] = pd.to_datetime(weather["Date"], format="%Y%m%d")

merged = impact_loc.merge(weather, left_on=["date_key", "grid_lat", "grid_lon"], right_on=["Date", "lat", "lng"], how='left')
merged = merged.sort_values(by="Date")

In [45]:
#impact_loc.to_hdf("impact_scores.h5", key='df', mode='a')

### Add Empty Rows for Missing Weather Data

In [46]:
left = merged.loc[pd.isna(merged["Date"])].sort_values("date_key")

In [47]:
# Add empty rows for impact scores that don't have any weather data
missing = left.loc[left.index.repeat(4)]

In [48]:
# Add empty rows for impact scores that have less than 4 weather timestamps
complete = merged.loc[pd.notna(merged["Date"])]
g = complete.groupby(["zip5", "date_key"])
miss3 = g.filter(lambda x: len(x) == 1)
miss2 = g.filter(lambda x: len(x) == 2)
miss1 = g.filter(lambda x: len(x) == 3)

In [49]:
# missing 3 timestamps
miss3.iloc[:, 5:] = np.NaN
add3 = miss3.loc[miss3.index.repeat(3)]
print(miss3.shape)
print(add3.shape)

(637, 123)
(1911, 123)


In [50]:
# missing 2 timestamps
add2 = miss2
add2.iloc[:, 5:] = np.NaN
print(miss2.shape)
print(add2.shape)

(182, 123)
(182, 123)


In [51]:
# missing 2 timestamp
miss1_ = miss1.iloc[:, 0:5].drop_duplicates(["date_key", "zip5"])
add1 = miss1.loc[miss1_.index.repeat(1)]
add1.iloc[:, 5:] = np.NaN
print(miss1.shape)
print(add1.shape)

(1092, 123)
(364, 123)


In [52]:
# Add missing value rows to dataframe
missing = missing.append(add1)
missing = missing.append(add2)
missing = missing.append(add3)

In [53]:
# Verify that each 
data = complete.append(missing)
gg = data.groupby(["zip5", "date_key"])
print((gg.filter(lambda x: len(x) < 4)).shape)
print((gg.filter(lambda x: len(x) == 4)).shape)
print((gg.filter(lambda x: len(x) > 4).shape))

(0, 123)
(394700, 123)
(0, 123)


### Impute Missing Weather Data

In [ ]:
# use haversine formula to calculate shortest distance over the earth's surface
# https://en.wikipedia.org/wiki/Haversine_formula

def latlong_distance(lat1, long1, lat2, long2):
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    lat_delta = lat2 - lat1
    long_delta = np.radians(long2 - long1)
    #a = sin^2(latitude delta / 2) + cos latitude 1 * cos latitude 2
    #* sin^2(longitude delta / 2)
    a = np.sin(lat_delta / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(long_delta / 2) ** 2

    #c = 2 * arcsin(a^0.5)
    c = 2 * np.arcsin(np.sqrt(a))
    
    #d = R * c
    R = 6371 #earth's radius = 6371km
    d = R * c
    return d

In [ ]:
def closest_n_search(latlong, latlongs, **kwargs):
    # find n closest latlongs to the given coordinates and corresponding weights
    # input = tuple of target coordinates, list of tuples [(lat, long)], n
    # output = dict, key = weights, values = tuple(lat, long)
    target_lat, target_long = latlong
    min_latlong = tuple()
    min_distance = float('inf')
    dist = {}
    
    # evaluate each lat long pairs
    for (lat, long) in latlongs:
        # only evaluate lat long coordinates within a certain range
        if np.abs(lat - target_lat) <= 10 and np.abs(long - target_long) <= 10:
            dist = latlong_distance(target_lat, target_long, lat, long)

            if len(min_latlong) < 1 or dist <= min_distance:
                min_latlong = (lat, long)
                min_distance = dist
    return min_latlong, min_distance

### Pre-calculate closest n neighbors and corresponding weights of all coordinates

In [ ]:
# Pre-compute n nearest neighbor based on distance
# Select unique lat long combinations from the data
latlongs = data[['grid_lat', 'grid_lon']].values
latlongs = [(ll[0], ll[1]) for ll in latlongs]
latlongs = list(set(latlongs))
len(latlongs)

latlong_matches = {}
for i in range(len(latlongs)):
    latlong_match = closest_n_search(latlongs[i], latlongs[:i] + latlongs[i + 1:])
    latlong_matches[latlongs[i]] = latlong_match
    # return dict
    # key = (target_lat, target_long), value = (n_lat, n_long, dist)
        
print("example of coordinates at a similar distance to multiple points, target point :(36.0, -86.5), closest neighbor:",\
latlong_matches[(36.0, -86.5)])

dist_closest_n = [i[1] for i in list(latlong_matches.values())]

print(f"Average distance with geo-imputing ref (km): {np.mean(dist_closest_n)}")
print(f"Min. distance with geo-imputing ref (km): {np.min(dist_closest_n)}")
print(f"Max. distance with geo-imputing ref (km): {np.max(dist_closest_n)}")

#"Distribution of distance to closest neighbor (km)", plt.hist(dist_closest_n, bins=[0,100,200,300,400,500,600,700,800])

In [ ]:
sns.set(style="darkgrid")
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
ax = sns.distplot(dist_closest_n,bins=9,kde=False)
ax.set(xlabel='Distance to closest FC (km)', ylabel='Count')
fig.suptitle("Histogram of distance to closest neighbor of FCs", fontsize=15)
#plt.show()
fig.savefig('Figure_3.png')

### Resize data to ensure there are data records for all dates, zip5 and time (4 times a day)

In [ ]:
for index, row in data.iterrows():
    f = (data['date_key'] == row['date_key']) & (data['zip5'] == row['zip5'])
    if pd.isna(row['Date']):
        data.at[index,'Date'] = row['date_key']
    if pd.isna(row['Time']):
        missing_times = list(set(time) - set(data[f]['Time']))
        data.at[index,'Time'] = missing_times[0]

### Drop columns with > 80% missing data

In [ ]:
data = data.drop(columns = ["ForecastRange","Categorical_Snow_surface",\
                              "Composite_reflectivity_entire_atmosphere",
                              "Graupel_snow_pellets_hybrid",\
                              "Graupel_snow_pellets_isobaric",\
                              "Snow_mixing_ratio_hybrid",\
                              "Snow_mixing_ratio_isobaric",\
                              'Geopotential_height_potential_vorticity_surface', \
                              'u_component_of_wind_potential_vorticity_surface',\
                              'v_component_of_wind_potential_vorticity_surface',\
                              'Categorical_Ice_Pellets_surface',\
                              'Ice_water_mixing_ratio_hybrid',\
                              'Ice_water_mixing_ratio_isobaric',\
                              'Precipitation_rate_surface',\
                              'Vertical_velocity_geometric_isobaric',\
                              'Ice_growth_rate_altitude_above_msl',\
                              'Land_sea_coverage_nearest_neighbor_land1sea0_surface',\
                              'Pressure_potential_vorticity_surface',\
                              'Temperature_potential_vorticity_surface',\
                              'Vertical_Speed_Shear_potential_vorticity_surface',\
                              'Rain_mixing_ratio_hybrid',\
                              'Total_cloud_cover_isobaric',\
                              'Cloud_mixing_ratio_hybrid',\
                              'Categorical_Freezing_Rain_surface',\
                              'Categorical_Rain_surface', 'Visibility_surface'])

### Spatial Imputation

In [ ]:
def spatial_impute(lat, long, date, time, colname, data):
    n_latlong, dist = latlong_matches[(lat, long)]
    #print('\n=================')
    #print("target point:" , "(" ,lat,long, "), closest neighbor:", n_latlong, ", distance:", dist)
    b = data[(data['grid_lat'] == n_latlong[0]) & (data['grid_lon'] == n_latlong[1])\
        & (data['date_key'] == date) & (data['Time'] == time)]
    #print('\nData of closet neighbor\n',b,'\n')
    #print(np.sum(b))
    if b.shape[0] == 0:
        output = np.nan
    else: 
        output = b[colname].values[0]
    #print('imputed value:', output, 'distance',dist)
    #print('=================\n')
    return output, dist

In [ ]:
# Code for all features to impute

cols_to_impute = ['Snow_depth_surface', 'Haines_Index_surface',\
'u_component_of_wind_altitude_above_msl', 'v_component_of_wind_altitude_above_msl',\
'Soil_temperature_depth_below_surface_layer', 'Temperature_altitude_above_msl',\
'Volumetric_Soil_Moisture_Content_depth_below_surface_layer', 'Field_Capacity_surface', \
'Water_equivalent_of_accumulated_snow_depth_surface', 'Wilting_Point_surface', \
'Field_Capacity_surface', "Water_equivalent_of_accumulated_snow_depth_surface","Wilting_Point_surface"]

#Find index of column to impute
for colname in cols_to_impute:
    #colname = cols_to_impute[0]
    colind = list(data.columns).index(colname)
    print('Impute for column', colname)

    for i in range(data.shape[0]):
        data_target = data.iloc[i,:]
        #print(data_target)
        if np.isnan(data_target[colname]): #If data is missing, we need to impute
            print(i, '=>impute')
            lat, long, date, time = data_target['grid_lat'], data_target['grid_lon'], data_target['date_key'], data_target['Time']
            i_value, dist = spatial_impute(lat, long, date, time, colname, data)

            #Sucessful spatial imputation is when the imputed value is non-nan and the distance to the closest neighbor is < 200
            #The code currently set imputed value to Nan if the closest neighbor does not have value. 
            #However, it still returns a real value even when the distance to the closest neighbor is > 200. Code below is to fix it:
            if dist > 200:
                i_value = np.nan

            #Put the imputed values and flags inside the original dataframe:
            data.iloc[i,-3] = 1 #Update the column "any_impute_col" value to 1
            data.iloc[i,colind] = i_value
        #else:
        #    print(i, '=> no impute')

#### Temporal Interpolation

In [ ]:
#data = pd.read_hdf('imputeddata.h5')

In [ ]:
# Time-series interpolation
pd.to_datetime(data['datetime'])
data.set_index('datetime')
data.head()
data_interpolate = data.groupby(['grid_lat', 'grid_lon']).apply(lambda x: x.interpolate(method='linear', limit_direction = 'both'))

#### Temporal Extrapolation

In [ ]:
extrapcols = []
cols = data_interpolate.columns
for col in cols:
    if (sum(data_interpolate[col].isna()) != 0) & (col != "impact_score"): 
        extrapcols.append(col)

In [ ]:
testdat = data_interpolate.copy()

In [ ]:
for col in extrapcols:
    zips = data_interpolate[pd.isna(data_interpolate[col])]["zip5"].unique()
    for zipcode in zips:
        print("for " + str(zipcode) + " column: " + col)
        datum = data_interpolate[data_interpolate["zip5"] == zipcode][col].copy().reset_index()
           
        # Function to curve fit to the data
        def func(x, a, b, c, d):
            return a * (x ** 3) + b * (x ** 2) + c * x + d

        # Initial parameter guess, just to kick off the optimization
        guess = (0.5, 0.5, 0.5, 0.5)

        # Create copy of data to remove NaNs for curve fitting
        fit_datum = datum.dropna()

        # Place to store function parameters for each column
        col_params = {}

        # Get x & y and fit curve
        x = fit_datum.index.astype(float).values
        y = fit_datum[col].values
        # Curve fit column and get curve parameters
        params = curve_fit(func, x, y, guess)
        # Store optimized parameters
        col_params[col] = params[0]

        # Extrapolate for the columns
        # Get the index values for NaNs in the column
        x = datum[pd.isnull(datum[col])].index.astype(float).values
        # Extrapolate those points with the fitted function
        datum[col][x] = func(x, *col_params[col])
        print("testdat before assignment", testdat.loc[testdat["zip5"] == zipcode, col])
        print("datum extrap", datum[col])
        idx = datum["index"]
        testdat[col][idx] = datum[col]
        print("testdat after assignment", testdat.loc[testdat["zip5"] == zipcode, col])
        print("\n")
        #data_interpolate.loc[data_interpolate["zip5"] == zipcode][col] = datum

In [ ]:
# clipping data to the right range
cols_to_clip_0= ['Snow_depth_surface',\
'Soil_temperature_depth_below_surface_layer', 'Temperature_altitude_above_msl',\
'Volumetric_Soil_Moisture_Content_depth_below_surface_layer','Field_Capacity_surface', \
"Water_equivalent_of_accumulated_snow_depth_surface","Wilting_Point_surface"]
cols_to_clip_2 = ['Haines_Index_surface']

In [ ]:
for col in cols_to_clip_0:
    testdat[col].clip(lower=0, inplace=True)

In [ ]:
for col in cols_to_clip_2:
    testdat[col].clip(lower=2, upper=6, inplace=True)
    testdat[col] = testdat[col].round()

In [ ]:
data = testdat

### Adding Features from External Data Scources

In [54]:
### Add state, region, and urban/suburban information from FAR codes dataset
data = data.sort_values(["zip5", 'date_key', 'Time'])
x = data['zip5']
# Read in Far codes
far = pd.read_excel('FARcodesZIPdata2010WithAKandHI.xlsx', sheet_name = 1)
cols = ['ZIP', 'density', 'state']
far = far[cols]
new_row = {'ZIP': 2722, 'density': 0, 'state': 'MA'}
far = far.append(new_row, ignore_index=True)

In [55]:
# Add density and state
data = data.merge(far, left_on = 'zip5', right_on = 'ZIP')
y = data['zip5']

In [56]:
# State to region dictionary
New_England_Northeast = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
Mid_Atlantic_Northeast = ['NJ', 'NY', 'PA']
East_North_Central_Midwest = ['IL', 'IN', 'MI', 'OH', 'WI'] 
West_North_Central_Midwest = ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
South_Atlantic_South = ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV']
East_South_Central_South = ['AL', 'KY', 'MS', 'TN']
West_South_Central_South = ['AR', 'LA', 'OK', 'TX']
Mountain_West = ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY']
Pacific_West = ['AK', 'CA', 'HI', 'OR', 'WA']

# Add region
data['Region'] = ['New England Northeast' if x in New_England_Northeast else
                 'Mid Atlantic Northeast' if x in Mid_Atlantic_Northeast else
                 'East North Central Midwest' if x in East_North_Central_Midwest else
                 'West North Central Midwest' if x in West_North_Central_Midwest else
                 'South Atlantic South' if x in South_Atlantic_South else
                 'East South Central South' if x in East_South_Central_South else
                 'West South Central South' if x in West_South_Central_South else
                 'Mountain West' if x in Mountain_West else
                 'Pacific West' for x in data['state']]
data = data.drop(["ZIP"], axis=1)

In [57]:
# Add suburban or urban
data['USR'] = ['Urban' if x > 3000 else 'Rural' if x < 1000 else 'Suburban' for x in data['density']]

In [58]:
# Add day of the week
data['Weekday'] = data['date_key'].dt.dayofweek
data['is_weekend'] = (data['date_key'].dt.dayofweek < 5).astype(int)

# Add date, month and year
data['day'] = data['date_key'].dt.day
data['month'] = data['date_key'].dt.month
data['year'] = data['date_key'].dt.year

In [59]:
# Add federal holidays
cal = calendar()
holidays = cal.holidays(start="2017-01-01", end="2019-12-31")

data['is_holiday'] = (data["date_key"].isin(holidays)).astype(int)

In [ ]:
# Add categorical variables for date and zip
data['year'] = pd.Categorical(data['year'])
data['month'] = pd.Categorical(data['month'])
data['day'] = pd.Categorical(data['day'])
data['zip5'] = pd.Categorical(data['zip5'])

In [ ]:
# Add binary variables for state, rural/urban, region, weekday, year, month, and day of week
cols = ['state', 'USR', 'Region', 'Weekday', 'year', 'month', 'day']
df = data[cols]
dataDummies = pd.get_dummies(df,drop_first=True)
data = pd.concat([data, dataDummies], axis = 1)
data = data.drop(['state', 'USR', 'Region', 'year', 'month', 'day'], axis = 1)

In [60]:
data

,date_key,zip5,impact_score,grid_lat,grid_lon,Date,Time,ForecastRange,x,y,...,density,state,Region,USR,Weekday,is_weekend,day,month,year,is_holiday
0,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,0.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
1,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,6.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
2,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,12.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
3,2017-01-01,2722,20.268081,41.5,-71.0,2017-01-01,18.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,6,0,1,1,2017,0
4,2017-01-02,2722,16.868994,41.5,-71.0,2017-01-02,0.0,0.0,578.0,97.0,...,0.000000,MA,New England Northeast,Rural,0,1,2,1,2017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394695,2019-12-30,98421,NaN,47.5,-122.5,2019-12-30,18.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,0,1,30,12,2019,0
394696,2019-12-31,98421,NaN,47.5,-122.5,2019-12-31,0.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0
394697,2019-12-31,98421,NaN,47.5,-122.5,2019-12-31,6.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0
394698,2019-12-31,98421,NaN,47.5,-122.5,2019-12-31,12.0,0.0,950.0,170.0,...,208.237785,WA,Pacific West,Rural,1,1,31,12,2019,0


### Store full dataset

In [61]:
data.to_hdf("full.h5", key='df', mode='a')

# Modeling

In [ ]:
#Load data
data = pd.read_hdf('full.h5')
data.head()
# Preprocessing
data = data.drop(['zip5'], axis = 1)
data = data[data['datetime'].dt.year < 2019]
data = data.drop(['x', 'y', 'any_impute_col', 'impute_row', 'datetime'], axis = 1)

### Training and Test Split

In [ ]:
#Split data to training and validation (based on percentage)

end_ind = int(0.7*data.shape[0])
trainData, testData = data.loc[0:end_ind,:], data.loc[end_ind+1:data.shape[0],:]
print(trainData.shape, testData.shape)

X_train = trainData.drop(["impact_score"], axis = 1)
X_test = testData.drop(["impact_score"], axis = 1)

Y_train = trainData["impact_score"]
Y_test = testData["impact_score"]
print('train data: ', X_train.shape, Y_train.shape, '\ntest data: ', X_test.shape, Y_test.shape)

### Regression

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train.values, Y_train.values)

# Make predictions using the testing set
y_pred_MLR_ori = regr.predict(X_test.values)

# The coefficients
#print('Coefficients: \n', regr.coef_)
# The mean squared error

y_pred_MLR = y_pred_MLR_ori.flatten()

# Clipping values
y_pred_MLR[y_pred_MLR<-1] = 0
y_pred_MLR[y_pred_MLR>35] = 35

print('MLR Error: %.2f'
      % mean_squared_error(Y_test.values, y_pred_MLR))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(Y_test.values, y_pred_MLR))

### Random Forest

In [ ]:
# Random Forest Regression modeling - Parameter Tuning: n_est
n_est = 40 #The number of trees in the forest.
max_d = 10 #Max depth of the tree
min_samples = 100 #The minimum number of samples required to be at a leaf node. 

n_est_dict_y_e = {}
n_est_dict_mse_e = {}

for n_est in range(40,220,40):
    m1 = RandomForestRegressor(n_estimators = n_est, max_depth = max_d, min_samples_leaf = min_samples)
    m1.fit(X_train,Y_train)
    y_pred_RF = m1.predict(X_test)
    n_est_dict_mse_e[n_est] = metrics.mean_squared_error(Y_test, y_pred_RF)
    n_est_dict_y_e[n_est] = y_pred_RF
    print("RF Error:", metrics.mean_squared_error(Y_test, y_pred_RF))

In [ ]:
# Random Forest Regression modeling - Parameter Tuning: max_d
n_est = 40 #The number of trees in the forest.
max_d = 10 #Max depth of the tree
min_samples = 100 #The minimum number of samples required to be at a leaf node. 

n_est_dict_y_d = {}
n_est_dict_mse_d = {}

for max_d in range(10,22,2):
    m1 = RandomForestRegressor(n_estimators = n_est, max_depth = max_d, min_samples_leaf = min_samples)
    m1.fit(X_train,Y_train)
    y_pred_RF = m1.predict(X_test)
    n_est_dict_mse_d[max_d] = metrics.mean_squared_error(Y_test, y_pred_RF)
    n_est_dict_y_d[max_d] = y_pred_RF
    print("RF Error:", metrics.mean_squared_error(Y_test, y_pred_RF))

#### Random Forest Hyperparameter Tuning

Varying number of trees (n_estimators)

n_estimators|max_d|MSE|
---|---|---|
40|10|43.69|
80|10|43.69|
120|10|43.79|
160|10|43.46|
200|10|43.75|


Varying minimum number of samples per leaf node (n_estimators)

n_estimators|max_d|MSE|
---|---|---|
40|10|43.70|
40|12|43.83|
40|14|42.69|
40|16|41.94|
40|18|43.46|
40|20|42.43|

In [ ]:
# Random Forest Regression modeling - Final modeling
n_est = 160 #The number of trees in the forest.
max_d = 16 #Max depth of the tree
min_samples = 100 #The minimum number of samples required to be at a leaf node. 

m1 = RandomForestRegressor(n_estimators = n_est, max_depth = max_d, min_samples_leaf = min_samples)
m1.fit(X_train,Y_train)
y_pred_RF = m1.predict(X_test)
print("RF Error:", metrics.mean_squared_error(Y_test, y_pred_RF))

#### Result Visualization of Regression and Random Forest

In [ ]:
#Join testing data: X_test, y_test, y_pred to a single dataframe for visualization
y_vals = X_test.copy()
y_vals['true_IS'], y_vals['pred_RF'], y_vals['pred_MLR'] = y_test, y_pred_RF, y_pred_MLR

y_vals = y_vals.reset_index()

In [ ]:
#Plot a certain zip code data. i.e. lat = 47, long = -122
zip_lat, zip_lon = 47, -122
data_zip = y_vals[(y_vals['lat']==zip_lat) & (y_vals['lng']==zip_lon)]

print(data_zip.shape)

fig, ax = plt.subplots(1,1,figsize=(15,5))
ax.plot(data_zip['true_IS'], label = 'True Value')
ax.plot(data_zip['pred_RF'], label = 'Random Forest')
ax.plot(data_zip['pred_MLR'], label = 'Regression')
ax.set(xlabel='Index', ylabel='Impact Score')
ax.legend()
fig.suptitle('lat '+str(zip_lat)+ ' & long ' + str(zip_lon), fontsize=15)

## CNN

In [ ]:
#reshaping functions Chitwan made
def rolling_window(a, window):
    a_copy = a.copy()
    shape = a_copy.shape[:-1] + (a_copy.shape[-1] - window + 1, window)
    strides = a_copy.strides + (a_copy.strides[-1],)
    return np.lib.stride_tricks.as_strided(a_copy, shape=shape, strides=strides)

def reshape_X(X, time_interval):
    XT = np.transpose(X.values)
    reshaped_X = np.transpose(rolling_window(XT, time_interval), (1,2,0))
    return reshaped_X

In [ ]:
# list of zips
zips = list(data.zip5.unique())
zips1 = zips[:23]
zips2 = zips[23:46]
zips3 = zips[46:69]
zips4 = zips[69:]
zipsx = [29172]

In [ ]:
#dictionary of predictions
predictions = {}

#list of rsme
rsme = []

#change zips to your set of zips
for z in zips2:
    lag = 28
    # Save model path
    checkpointer_path = f"./models1/{z}.hdf5"
    
    #subsetting data for model
    test = data[data['zip5'] == z]
    test = test.drop(['zip5', 'datetime'], axis =1)
    test_17_18 = test[test['year_2019'] != 1]
    
    #19 test
    test_19 = test[test['year_2019'] == 1]
    test_19 = test_17_18[-lag+1:].append(test_19, ignore_index = True)
    test_19 = test_19.drop(['impact_score'], axis = 1)
    test_19 = reshape_X(test_19, lag)
    
    #final y list
    y = test_17_18['impact_score']
    y = y[lag-1:]
    
    #final X
    X = test_17_18.drop(['impact_score'], axis = 1)
    X = reshape_X(X, lag)
    
    #data split
    test_size = int(0.33 * X.shape[0])
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    
    checkpointer = ModelCheckpoint(checkpointer_path, monitor='val_loss', verbose=1, save_best_only=True)
    
    #model architecture
    model = Sequential((
        Convolution1D(input_shape=(lag, 173), 
                      kernel_size=4, activation="relu", filters=64),
        MaxPooling1D(),    
        Dropout(0.2),
        Convolution1D(input_shape=(lag-4+1, 173), 
                      kernel_size=4, activation="relu", filters=64),
        MaxPooling1D(),    
        Dropout(0.3),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1),
    ))
    opt = Adam(lr=0.0005)
    
    #output of model while running
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])
    model.summary()
    
    #stop to prevent overfitting
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    
    #model fit and test
    val = model.fit(X_train, y_train, epochs=100, batch_size=5, validation_data=(X_test, y_test), callbacks=[checkpointer])
    pred = model.predict(X_test)
    testScore = math.sqrt(mean_squared_error(y_test,pred))
    rsme.append(testScore)
    
    #model predictions
    model = load_model(f"./models1/{z}.hdf5")
    pred_19 = model.predict(test_19)
    predictions[z] = pred_19